In [1]:
import numpy as np

from mvpa2.datasets.base import Dataset
from mvpa2.algorithms.hyperalignment import Hyperalignment

In [169]:
# Define a function to create a random 3D rotation matrix
def rand_rotation():
    x_rotation = np.random.uniform(0,2*np.pi,1)
    Rx = [[1, 0, 0], 
          [0, np.cos(x_rotation), -np.sin(x_rotation)],
          [0, np.sin(x_rotation), np.cos(x_rotation)]]

    y_rotation = np.random.uniform(0,2*np.pi,1)
    Ry = [[np.cos(y_rotation), 0, np.sin(y_rotation)], 
          [0, 1, 0],
          [-np.sin(y_rotation), 0, np.cos(y_rotation)]]

    z_rotation = np.random.uniform(0,2*np.pi,1)
    Rz = [[np.cos(z_rotation), -np.sin(z_rotation), 0], 
          [np.sin(z_rotation), np.cos(z_rotation), 0],
          [0, 0, 1]]
    
    return np.dot(np.dot(Rx,Ry),Rz)

In [170]:
# Create fake rotated data
D = np.random.uniform(0,1,[20,3])
datasets = [Dataset(D)]
rotations = [np.identity(3)]
for i in range(5):
    R = rand_rotation()
    rotations.append(R)
    datasets.append(Dataset(np.dot(D,R)))

In [171]:
# Initialize a hyperalingment object
hyper_aligner = Hyperalignment()

In [172]:
# Train our hyperalingment model
hyper_aligner.train(datasets)

In [173]:
# Collect our mappings from fitting hyperalignment
mappers = hyper_aligner(datasets)

In [174]:
# Apply the hyper-alignment projection to transform the data
new_datasets = []
for dataset, mapper in zip(datasets, mappers):
    new_datasets.append(mapper.forward(dataset))
    
# Note: this above is equivalent to the following one liner
new_datasets = [mapper.forward(dataset) for dataset, mapper in zip(datasets, mappers)]

In [178]:
# See if alignment worked
ex1 = new_datasets[0]
ex2 = new_datasets[3]
print ex1.samples - ex2.samples

[[-5.91193761e-15  1.55431223e-15  6.66133815e-16]
 [ 1.33226763e-15 -3.33066907e-15  3.77475828e-15]
 [-3.10862447e-15  3.66373598e-15 -3.33066907e-15]
 [-4.44089210e-16  2.44249065e-15 -2.99760217e-15]
 [-1.66533454e-16  2.22044605e-16 -2.22044605e-16]
 [ 2.31759056e-15 -2.88657986e-15  1.77635684e-15]
 [-3.21964677e-15  1.77635684e-15 -1.33226763e-15]
 [ 1.11022302e-16 -1.11022302e-16 -9.99200722e-16]
 [ 7.49400542e-16 -2.66453526e-15  2.22044605e-15]
 [ 1.05471187e-15 -1.30971622e-16 -1.11022302e-15]
 [ 2.33146835e-15 -3.44169138e-15  2.66453526e-15]
 [-8.88178420e-16  2.38697950e-15 -3.44169138e-15]
 [ 2.53963517e-15  2.22044605e-16 -1.60982339e-15]
 [ 3.77475828e-15  9.99200722e-16 -4.55191440e-15]
 [ 3.77475828e-15 -2.88657986e-15  1.27675648e-15]
 [-8.88178420e-16 -1.11022302e-15  8.88178420e-16]
 [-7.77156117e-15  1.55431223e-15  1.33226763e-15]
 [ 4.44089210e-15 -5.32907052e-15  4.44089210e-15]
 [ 2.77555756e-15 -2.55351296e-15  9.99200722e-16]
 [-1.33226763e-15  2.22044605e-

In [175]:
# From here: try classification between subjects, in aligned space